In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter

# Custom KNN class
class CustomKNN:
    def __init__(self, k=3, weight='uniform'):
        self.k = k
        self.weight = weight

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        distances = [np.sqrt(np.sum((x - x_train) ** 2)) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        if self.weight == 'uniform':
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        elif self.weight == 'distance':
            weights = [1 / (d + 1e-6) for d in distances[k_indices]]
            weighted_counts = Counter(k_nearest_labels)
            weighted_votes = {label: sum(weights[i] for i, l in enumerate(k_nearest_labels) if l == label) for label in set(k_nearest_labels)}
            return max(weighted_votes, key=weighted_votes.get)

# Custom train-test split function
def custom_train_test_split(X, y, test_size=0.2, random_state=None):
    num_samples = X.shape[0]
    num_test_samples = int(test_size * num_samples)
    if random_state is not None:
        np.random.seed(random_state)
    indices = np.random.permutation(num_samples)
    test_indices = indices[:num_test_samples]
    train_indices = indices[num_test_samples:]
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    return X_train, X_test, y_train, y_test

# Simplified GridSearch function
def custom_grid_search(X_train, y_train, param_grid, model_class):
    best_accuracy = 0.0
    best_params = {}
    for params in param_grid:
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = np.mean(y_pred == y_test)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params
    return best_params, best_accuracy

# Load Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Custom train-test split
X_train, X_test, y_train, y_test = custom_train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for GridSearch
param_grid = [
    {'k': [1, 3, 5, 7], 'weight': ['uniform', 'distance']}
]

# Perform GridSearch
best_params, best_accuracy = custom_grid_search(X_train, y_train, param_grid, CustomKNN)
print("Best Parameters:", best_params)
print("Best Accuracy:", best_accuracy)
